# Designing a 4-way demultiplexer


This notebook is a larger excercise where we are going to design a 4-way drop filter as depcited in the following image.

<img src="_images/demux.png" width=800>

with the following transmission where the resonnances are separated by 200 GHz.

<img src="_images/demux_trans.png" width=800 >


As demonstrated in the previous notebook you will have to use PlaceAndAutoRoute with correct values for the **cells**, **links** and **transformations**. 

## Cells

There are 4 components involved here:

- Grating Coupler
- Splitter 
- Ring Resonators (each tuned to a different resonance frequency)
- Terminations for the unused drop port of the ring.

Each of those components have a Layout and a Model

Let us import everything we need for that first.

In [ ]:
# Fix paths so that relative imports work in ipython 
import os, sys
sys.path.append('.')
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 6)
from technologies import silicon_photonics
from ipkiss3 import all as i3
import numpy as np
import pylab as plt

from picazzo3.traces.wire_wg import WireWaveguideTemplate
from picazzo3.fibcoup.curved import FiberCouplerCurvedGrating
from picazzo3.apertures.basic import WireWgAperture
from support_files.mmi import SimpleMMI
from support_files.ring import SimpleRing
from picazzo3.routing.place_route import PlaceAndAutoRoute

wavelengths =  np.linspace(1.52, 1.58, 4001)

### Grating coupler

For the grating coupler, we use the Picazzo libary.

In [ ]:
grating_coupler = FiberCouplerCurvedGrating()
grating_coupler_layout = grating_coupler.Layout()
grating_cm = grating_coupler.CircuitModel(center_wavelength=1.55, bandwidth_3dB=0.06, peak_transmission=0.60**0.5, reflection=0.01**0.5)
# Plot Layout
grating_coupler_layout.visualize(annotate=True)
# Plot model
S = grating_cm.get_smatrix(wavelengths=wavelengths)
plt.plot(wavelengths, 20 * np.log10(np.abs(S['vertical_in', 'out'])), label='coupling')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['out', 'out'])), label='reflection')
plt.legend()

### Splitter
Also for the splitter we use the Picazzo library 

In [ ]:
splitter = SimpleMMI()
splitter_layout = splitter.Layout(length=14.1428571429, waveguide_spacing=2.1)
splitter_cm = splitter.CircuitModel(straight_coupling1=0.45**0.5)

# Plot Layout
splitter_layout.visualize(annotate=True)
# Plot model
S = splitter_cm.get_smatrix(wavelengths=wavelengths)
plt.plot(wavelengths, 20 * np.log10(np.abs(S['in', 'out1'])), label='thru1')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['in', 'out2'])), label='thru2')
plt.legend()

### Rings

We want ring resonators that are spaced by 200 GHz. To make your life easier we have created our own ring class that uses RingRect180DropFilter but specializes it to take resonance frequency as a property. Internally the ring class takes a look at the effective index of the waveguide and calculates its bend radius automatically. If you are interested feel free to take a look, here we will just use the class.

<font color="black">
Ring: [ring.py](/edit/base_pcell_views_properties/support_files/ring.py)

As we have a 4 rings it is most convenient to use a for loop and fill an array with the ring cells.

In [ ]:
spacing_ghz = 200
spacing_um = (spacing_ghz*1e9 * (1.55*1e-6)**2 / 3e8)*1e6 #Calculate the spacing in um
res_w = 1.550 + spacing_um * np.array([0.0, 1.0, 2.0, 3.0])
rings = []
r_layouts = []
for nr, r in enumerate(res_w):
    ring=SimpleRing(resonance_wavelength=r)
    rings.append(ring)
    r_layout = ring.Layout()
    r_layouts.append(r_layout)
    if nr ==0:
        # Visualize Layout (only the first one)
        r_layout.visualize(annotate=True)
        r_cm = ring.CircuitModel()
        # Simulate ring (only the first one)
        S = r_cm.get_smatrix(wavelengths=wavelengths)
        plt.plot(wavelengths, 20 * np.log10(np.abs(S['in1', 'out1'])), label='ring{}'.format(nr))
        plt.ylim([-80,1])
    print "Ring created with resonance frequency  {}".format(r)


### Terminations

To avoid back-reflections into the ring is better to put terminations on the unused ports. 
Also here we use Picazzo 

In [ ]:
wt = WireWaveguideTemplate()
wt.Layout(core_width = 0.9,
          cladding_width = 2 * i3.TECH.WG.WIRE_WIDTH + 0.9)
wwa = WireWgAperture(aperture_trace_template = wt)
wwa_lay = wwa.Layout()
wwa_cm = wwa.CircuitModel(reflection=0.001)

# Plot Layout
wwa_lay.visualize(annotate=True)
# Plot model
S = wwa_cm.get_smatrix(wavelengths=wavelengths)
plt.plot(wavelengths, 20 * np.log10(np.abs(S['in','in'])), label='reflection')


## Making the circuit

Now is the time to make the full circuit using PlaceAndAutoRoute, which requires passing a dict of **cells**, **links** and **transformations** to pass to PlaceAndAutoRoute later. 

#### Vectormatchtransform. 

To calculate our transformations we can use  `i3.vector_match_transform`, which can use a bit more explaination. As the name implies, this transformation matches two vectors together. For instance:


`i3.vector_match_transform(grating_coupler_layout.ports["out"], i3.Vector(position=(0,0), angle=180.0))`

When applied to the grating coupler, it will place the out port of the grating coupler at position (0,0) and align the port to 180.0 degrees, meaning that it would accept a waveguide coming from a 180 angle. The port itself would then point at 0.0 degrees.

#### Up to you

Now it is up to you to fill **cells**, **links** and **transformations**. We have already put the components in there - its up to you to complete the circuit. 


<font color="black">
Solution: [exercise_3.py](/edit/base_pcell_views_properties/support_files/place_and_auto_route/exercise_3.py)

In [ ]:
# Making the circuit
cells = {"gc_in"     : grating_coupler,
         "gc_ref"    : grating_coupler,
         "gc_pass"   : grating_coupler,
         "splitter"  : splitter
         }

# Update the cells dict with all rings, grating couplers, and terminations
for cnt, r in enumerate(rings, 1):
    cells["ring{}".format(cnt)] = rings[cnt - 1]
    cells["gc_out{}".format(cnt)] = grating_coupler
    cells["term_out{}".format(cnt)] = wwa
    
         
links = [# input coupler to splitter
         ("gc_in:out",      "splitter:in"), 
         
         # reference output to splitter output
         ("gc_ref:out",     "splitter:out1")]
         
d = 100.0
transformations = dict()
transformations['gc_in'] = i3.vector_match_transform(grating_coupler_layout.ports["out"], i3.Vector(position=(0,0), angle=180.0))
transformations['splitter'] = i3.vector_match_transform(splitter_layout.ports["in"], i3.Vector(position=(d/2,0), angle=0.0))

for cnt, r in enumerate(rings, 1):
    transformations["ring{}".format(cnt)] = i3.vector_match_transform(r_layouts[cnt-1].ports["in1"], i3.Vector(position=(cnt*d,d/2), angle=0.0))
    transformations["gc_out{}".format(cnt)] = i3.vector_match_transform(grating_coupler_layout.ports["out"], i3.Vector(position=(cnt*d-10,d), angle=90.0))
    transformations["term_out{}".format(cnt)] = i3.Rotation(rotation=90.0,rotation_center=wwa_lay.ports["in"]) + \
        i3.vector_match_transform(wwa_lay.ports["in"],
        r_layouts[cnt-1].ports["in2"].transform_copy(transformations["ring{}".format(cnt)])) + \
        i3.Translation(translation=(10.0,10.0))



transformations['gc_ref'] = i3.vector_match_transform(grating_coupler_layout.ports["out"], i3.Vector(position=(5*d,-d/2), angle=0.0))
transformations['gc_pass'] = i3.vector_match_transform(grating_coupler_layout.ports["out"], i3.Vector(position=(5*d,d/2), angle=0.0))  

## Plot the layout and simulate the circuit

We now use PlaceAndAutoRoute to create the circuit and do the circuit simulation.

In [ ]:
# Create the PCell
my_circuit = PlaceAndAutoRoute(child_cells=cells,
                               links=links)

# Create Layout
layout = my_circuit.Layout(child_transformations=transformations)
layout.visualize(annotate=True)
layout.write_gdsii("full_circuit.gds")


# Flatten design

If the you pick resonance frequencies that lead to a off-grid ports it is safest to flatten your entire design as different GDSII viewers do rounding in a slightly different way. The following lines of code show you how to do that.

In [ ]:

top_cell_layout_flat = layout.layout.flat_copy()
top_layout_flat = i3.LayoutCell().Layout(elements=top_cell_layout_flat)
top_layout_flat.write_gdsii("full_circuit_flat.gds")


### Simulate the circuit

You can now run a circuit simulation.

Note: If the circuit is wrong or uncomplete, your circuit simulation will be too.

In [ ]:
cm = my_circuit.CircuitModel()
wavelengths =  np.linspace(1.52, 1.58, 4001)
S = cm.get_smatrix(wavelengths=wavelengths)

plt.figure()
plt.plot(wavelengths, 20 * np.log10(np.abs(S['gc_in_vertical_in', 'gc_pass_vertical_in'])), label='pass')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['gc_in_vertical_in', 'gc_out1_vertical_in'])), label='out1')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['gc_in_vertical_in', 'gc_out2_vertical_in'])), label='out2')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['gc_in_vertical_in', 'gc_out3_vertical_in'])), label='out3')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['gc_in_vertical_in', 'gc_out4_vertical_in'])), label='out4')
plt.plot(wavelengths, 20 * np.log10(np.abs(S['gc_in_vertical_in', 'gc_ref_vertical_in'])), label='ref')
plt.title("Circuit transmission (power)")
plt.xlabel("Wavelength ($\mu m$)")
plt.ylabel("Transmission [dB]")
plt.ylim([-80,0])
plt.xlim([1.52,1.60])
for w in res_w:
    plt.axvline(x=w)
plt.legend()
plt.show()

### Additional Exercise

#### Create a 8-way multiplexer.

Try to create a 8-way demultiplexer with a channel spacing of 100 GHz.

<img src="_images/8mux.png" width=800>
<img src="_images/8way_trans.png" width=800>




**HINT** Best is to make a copy of this notebook and make the modifications in there (file-> Make a copy)

Solution: [exercise_4.py](/edit/base_pcell_views_properties/support_files/place_and_auto_route/exercise_4.py)
